In [56]:
import requests
import pandas as pd
import config
import json
import time
import gmaps

# Scrape Coordinates

* Notebook uses Google Geocode API to get info about ~2000 city coordinates (latitude, longitude) where startups are located

* Free monthly limit of about 40,000 API calls from automatic free $200 credit/month

* CSV and JSON output files of city and coordinate data - CSV format for use in other notebooks with pandas, and JSON format for preliminary setup and test runs of geographical mapping in Leaflet

In [24]:
#test that everything is working correctly on an example request
base_url = "https://maps.googleapis.com/maps/api/geocode/json?address="

city = "San Francisco"
state_code = "CA"
country_code = "USA"

response = requests.get("{}{}+{}+{}&key={}".format(base_url, city, state_code, country_code, config.API_KEY))

print(json.dumps(response.json(), indent=4))
print(response.json()["results"][0]["geometry"]["location"])

{
    "results": [
        {
            "address_components": [
                {
                    "long_name": "San Francisco",
                    "short_name": "SF",
                    "types": [
                        "locality",
                        "political"
                    ]
                },
                {
                    "long_name": "San Francisco County",
                    "short_name": "San Francisco County",
                    "types": [
                        "administrative_area_level_2",
                        "political"
                    ]
                },
                {
                    "long_name": "California",
                    "short_name": "CA",
                    "types": [
                        "administrative_area_level_1",
                        "political"
                    ]
                },
                {
                    "long_name": "United States",
                    "short_name": "US",
           

In [31]:
#read in CSV containing unique city, state, and country data
valid_locations_df = pd.read_csv("./geographical_data/all_places.csv")
valid_locations_df

,country_code,state_code,city
0,USA,NY,New York
1,USA,IL,Champaign
2,USA,CA,Los Angeles
3,USA,NJ,Iselin
4,USA,MD,Woodbine
...,...,...,...
2168,CAN,ON,Paris
2169,USA,FL,Atlantic Beach
2170,USA,OR,Salem
2171,USA,WA,Auburn


In [32]:
#initialize copy of data frame to store data
coords_df = valid_locations_df.copy()
coords_df["Latitude"] = None
coords_df["Longitude"] = None
coords_df

,country_code,state_code,city,Latitude,Longitude
0,USA,NY,New York,None,None
1,USA,IL,Champaign,None,None
2,USA,CA,Los Angeles,None,None
3,USA,NJ,Iselin,None,None
4,USA,MD,Woodbine,None,None
...,...,...,...,...,...
2168,CAN,ON,Paris,None,None
2169,USA,FL,Atlantic Beach,None,None
2170,USA,OR,Salem,None,None
2171,USA,WA,Auburn,None,None


In [33]:
#write for loop to send requests each second, and append data to a copy of the read dataframe
for i in range(len(coords_df)):
    city = coords_df.iloc[i]["city"]
    state = coords_df.iloc[i]["state_code"]
    country = coords_df.iloc[i]["country_code"]
    try:
        response = requests.get("{}{}+{}+{}&key={}".format(base_url, city, state, country, config.API_KEY))
        results = response.json()
        print(results["results"][0]["geometry"]["location"])
        lat = results["results"][0]["geometry"]["location"]["lat"]
        lng = results["results"][0]["geometry"]["location"]["lng"]
        time.sleep(1)
        coords_df["Latitude"][i] = lat
        coords_df["Longitude"][i] = lng
        print("Working response for {}, {}, {} with results: {}, {}".format(city, state, country, lat, lng))
    except:
        print("Issue with {}, {}, {}".format(city, state, country))


{'lat': 40.7127753, 'lng': -74.0059728}
Working response for New York, NY, USA with results: 40.7127753, -74.0059728
{'lat': 40.1164204, 'lng': -88.2433829}
Working response for Champaign, IL, USA with results: 40.1164204, -88.2433829
{'lat': 34.0522342, 'lng': -118.2436849}
Working response for Los Angeles, CA, USA with results: 34.0522342, -118.2436849
{'lat': 40.5753817, 'lng': -74.3223703}
Working response for Iselin, NJ, USA with results: 40.5753817, -74.3223703
{'lat': 39.3738173, 'lng': -77.0569038}
Working response for Woodbine, MD, USA with results: 39.3738173, -77.0569038
{'lat': 45.415734, 'lng': -122.7163273}
Working response for Lake Oswego, OR, USA with results: 45.415734, -122.7163273
{'lat': 37.7749295, 'lng': -122.4194155}
Working response for San Francisco, CA, USA with results: 37.7749295, -122.4194155
{'lat': 47.6101497, 'lng': -122.2015159}
Working response for Bellevue, WA, USA with results: 47.6101497, -122.2015159
{'lat': 43.0730517, 'lng': -89.4012302}
Working 

Working response for Santa Clara, CA, USA with results: 37.3541079, -121.9552356
{'lat': 38.84622359999999, 'lng': -77.30637329999999}
Working response for Fairfax, VA, USA with results: 38.84622359999999, -77.30637329999999
{'lat': 42.3736158, 'lng': -71.10973349999999}
Working response for Cambridge, MA, USA with results: 42.3736158, -71.10973349999999
{'lat': 30.267153, 'lng': -97.7430608}
Working response for Austin, TX, USA with results: 30.267153, -97.7430608
{'lat': 39.9611755, 'lng': -82.99879419999999}
Working response for Columbus, OH, USA with results: 39.9611755, -82.99879419999999
{'lat': 45.6956723, 'lng': -121.8845241}
Working response for Stevenson, WA, USA with results: 45.6956723, -121.8845241
{'lat': 39.2903848, 'lng': -76.6121893}
Working response for Baltimore, MD, USA with results: 39.2903848, -76.6121893
{'lat': 42.3709299, 'lng': -71.1828321}
Working response for Watertown, MA, USA with results: 42.3709299, -71.1828321
{'lat': 33.7514966, 'lng': -84.7477136}
Wor

{'lat': 48.7519112, 'lng': -122.4786854}
Working response for Bellingham, WA, USA with results: 48.7519112, -122.4786854
{'lat': 38.2526647, 'lng': -85.7584557}
Working response for Louisville, KY, USA with results: 38.2526647, -85.7584557
{'lat': 38.8338816, 'lng': -104.8213634}
Working response for Colorado Springs, CO, USA with results: 38.8338816, -104.8213634
{'lat': 27.950575, 'lng': -82.4571776}
Working response for Tampa, FL, USA with results: 27.950575, -82.4571776
{'lat': 33.6845673, 'lng': -117.8265049}
Working response for Irvine, CA, USA with results: 33.6845673, -117.8265049
{'lat': 40.0992294, 'lng': -83.1140771}
Working response for Dublin, OH, USA with results: 40.0992294, -83.1140771
{'lat': 39.9205411, 'lng': -105.0866504}
Working response for Broomfield, CO, USA with results: 39.9205411, -105.0866504
{'lat': 41.9750287, 'lng': -88.0072909}
Working response for Itasca, IL, USA with results: 41.9750287, -88.0072909
{'lat': 39.327962, 'lng': -120.1832533}
Working respo

Working response for Laguna Hills, CA, USA with results: 33.5916433, -117.6986604
{'lat': 21.3069444, 'lng': -157.8583333}
Working response for Honolulu, HI, USA with results: 21.3069444, -157.8583333
{'lat': 28.0780718, 'lng': -82.7637127}
Working response for Palm Harbor, FL, USA with results: 28.0780718, -82.7637127
{'lat': 34.1477849, 'lng': -118.1445155}
Working response for Pasadena, CA, USA with results: 34.1477849, -118.1445155
{'lat': 38.3242355, 'lng': -85.4724593}
Working response for Crestwood, KY, USA with results: 38.3242355, -85.4724593
{'lat': 35.2270869, 'lng': -80.8431267}
Working response for Charlotte, NC, USA with results: 35.2270869, -80.8431267
{'lat': 42.0044757, 'lng': -87.73005940000002}
Working response for Lincolnwood, IL, USA with results: 42.0044757, -87.73005940000002
{'lat': 38.8942786, 'lng': -77.43109919999999}
Working response for Chantilly, VA, USA with results: 38.8942786, -77.43109919999999
{'lat': 41.0234304, 'lng': -74.06652969999999}
Working res

{'lat': 38.440429, 'lng': -122.7140548}
Working response for Santa Rosa, CA, USA with results: 38.440429, -122.7140548
{'lat': 37.8590937, 'lng': -122.4852507}
Working response for Sausalito, CA, USA with results: 37.8590937, -122.4852507
{'lat': 48.4284207, 'lng': -123.3656444}
Working response for Victoria, BC, CAN with results: 48.4284207, -123.3656444
{'lat': 41.8397865, 'lng': -87.95355339999999}
Working response for Oak Brook, IL, USA with results: 41.8397865, -87.95355339999999
{'lat': 40.6781784, 'lng': -73.9441579}
Working response for Brooklyn, NY, USA with results: 40.6781784, -73.9441579
{'lat': 40.8270448, 'lng': -73.97569399999999}
Working response for Edgewater, NJ, USA with results: 40.8270448, -73.97569399999999
{'lat': 40.5939636, 'lng': -74.6049061}
Working response for Bridgewater, NJ, USA with results: 40.5939636, -74.6049061
{'lat': 39.6477653, 'lng': -104.9877597}
Working response for Englewood, CO, USA with results: 39.6477653, -104.9877597
{'lat': 49.2488091, '

...

Working response for Quakertown, PA, USA with results: 40.4417682, -75.3415667
{'lat': 34.1397292, 'lng': -118.0353449}
Working response for Arcadia, CA, USA with results: 34.1397292, -118.0353449
{'lat': 41.18454149999999, 'lng': -73.1331651}
Working response for Stratford, CT, USA with results: 41.18454149999999, -73.1331651
{'lat': 40.3561805, 'lng': -80.1100556}
Working response for Bridgeville, PA, USA with results: 40.3561805, -80.1100556
{'lat': 41.4768217, 'lng': -81.8497567}
Working response for Rocky River, OH, USA with results: 41.4768217, -81.8497567
{'lat': 42.4795019, 'lng': -79.3339324}
Working response for Dunkirk, NY, USA with results: 42.4795019, -79.3339324
{'lat': 41.12086679999999, 'lng': -88.835352}
Working response for Streator, IL, USA with results: 41.12086679999999, -88.835352
{'lat': 41.2306979, 'lng': -73.064036}
Working response for Milford, CT, USA with results: 41.2306979, -73.064036
{'lat': 46.73238749999999, 'lng': -117.000165}
Working response for Mosc

{'lat': 40.7998227, 'lng': -73.6509622}
Working response for Roslyn, NY, USA with results: 40.7998227, -73.6509622
{'lat': 41.5623209, 'lng': -72.6506488}
Working response for Middletown, CT, USA with results: 41.5623209, -72.6506488
{'lat': 37.8806341, 'lng': -84.5729961}
Working response for Nicholasville, KY, USA with results: 37.8806341, -84.5729961
{'lat': 35.7234734, 'lng': -79.4622431}
Working response for Siler City, NC, USA with results: 35.7234734, -79.4622431
{'lat': 41.91252859999999, 'lng': -88.13479269999999}
Working response for Carol Stream, IL, USA with results: 41.91252859999999, -88.13479269999999
{'lat': 40.2005472, 'lng': -74.03141649999999}
Working response for Neptune, NJ, USA with results: 40.2005472, -74.03141649999999
{'lat': 40.3409251, 'lng': -76.4113497}
Working response for Lebanon, PA, USA with results: 40.3409251, -76.4113497
{'lat': 42.9133602, 'lng': -85.7053085}
Working response for Wyoming, MI, USA with results: 42.9133602, -85.7053085
{'lat': 41.506

{'lat': 48.5342662, 'lng': -123.0171242}
Working response for Friday Harbor, WA, USA with results: 48.5342662, -123.0171242
{'lat': 46.2305049, 'lng': -119.0922316}
Working response for Pasco, WA, USA with results: 46.2305049, -119.0922316
{'lat': 30.4548451, 'lng': -97.6222665}
Working response for Pflugerville, TX, USA with results: 30.4548451, -97.6222665
{'lat': 41.6883646, 'lng': -93.7925199}
Working response for Grimes, IA, USA with results: 41.6883646, -93.7925199
{'lat': 36.9759345, 'lng': -82.5757108}
Working response for Wise, VA, USA with results: 36.9759345, -82.5757108
{'lat': 25.7059354, 'lng': -80.35227499999999}
Working response for Sunset, FL, USA with results: 25.7059354, -80.35227499999999
{'lat': 44.7077106, 'lng': -100.0617352}
Working response for Onida, SD, USA with results: 44.7077106, -100.0617352
{'lat': 27.9462615, 'lng': -82.2420317}
Working response for Valrico, FL, USA with results: 27.9462615, -82.2420317
{'lat': 41.878711, 'lng': -71.38255579999999}
Work

Working response for Cedar City, UT, USA with results: 37.67747689999999, -113.0618931
{'lat': 38.403903, 'lng': -96.18166260000001}
Working response for Emporia, KS, USA with results: 38.403903, -96.18166260000001
{'lat': 38.4192496, 'lng': -82.44515400000002}
Working response for Huntington, WV, USA with results: 38.4192496, -82.44515400000002
{'lat': 42.587915, 'lng': -72.5994104}
Working response for Greenfield, MA, USA with results: 42.587915, -72.5994104
{'lat': 44.01690139999999, 'lng': -107.9553721}
Working response for Worland, WY, USA with results: 44.01690139999999, -107.9553721
{'lat': 35.8968236, 'lng': -77.5358049}
Working response for Tarboro, NC, USA with results: 35.8968236, -77.5358049
{'lat': 40.9414874, 'lng': -73.9593041}
Working response for Cresskill, NJ, USA with results: 40.9414874, -73.9593041
{'lat': 41.8939617, 'lng': -87.81984779999999}
Working response for River Forest, IL, USA with results: 41.8939617, -87.81984779999999
{'lat': 34.1805403, 'lng': -118.90

{'lat': 41.1073183, 'lng': -73.7959666}
Working response for Hawthorne, NY, USA with results: 41.1073183, -73.7959666
{'lat': 39.091449, 'lng': -84.4957757}
Working response for Newport, KY, USA with results: 39.091449, -84.4957757
{'lat': 35.0524699, 'lng': -118.1739645}
Working response for Mojave, CA, USA with results: 35.0524699, -118.1739645
{'lat': 34.78384090000001, 'lng': -86.5722237}
Working response for Normal, AL, USA with results: 34.78384090000001, -86.5722237
{'lat': 45.7762463, 'lng': -111.1770945}
Working response for Belgrade, MT, USA with results: 45.7762463, -111.1770945
{'lat': 46.15026599999999, 'lng': -67.6033952}
Working response for Woodstock, NB, CAN with results: 46.15026599999999, -67.6033952
{'lat': 41.84556509999999, 'lng': -71.886358}
Working response for Dayville, CT, USA with results: 41.84556509999999, -71.886358
{'lat': 29.9891052, 'lng': -97.87722670000001}
Working response for Kyle, TX, USA with results: 29.9891052, -97.87722670000001
{'lat': 40.8409

In [47]:
#check that dataframe looks correct
#add info if necessary, or drop items
coords_df
missing_vals = coords_df["Longitude"].isna()
print(coords_df[missing_vals])

     country_code state_code    city Latitude Longitude
1302          CAN         ON   Delta     None      None
1751          CAN         ON  Bolton     None      None


In [49]:
#only two missing values, so I manually got the values for latitude/longitude

#for Bolton, Ontario, Canada:
coords_df["Latitude"][1751] = 43.873021
coords_df["Longitude"][1751] = -79.728130

#for Delta, Ontario, Canada:
coords_df["Latitude"][1302] = 44.608509
coords_df["Longitude"][1302] = -76.123042

#check that all missing values have been filled
missing_vals = coords_df["Longitude"].isna()
print(coords_df[missing_vals])

Empty DataFrame
Columns: [country_code, state_code, city, Latitude, Longitude]
Index: []


In [52]:
#output data from new data frame containing latitudes and longitudes
#to csv in this cell
coords_df.to_csv("./geographical_data/coord_locations.csv", index=False)

In [55]:
#output data to json in this cell - to use for testing Leaflet maps
coords_df.to_json("./geographical_data/coord_locations.json", orient="index")

In [57]:
#look at distribution of cities around the world using gmaps!

# Configure gmaps
gmaps.configure(api_key=config.API_KEY)

#set locations (lat, long) data for the map
locations = coords_df.loc[:, ["Latitude", "Longitude"]]

# Plot Heatmap
fig = gmaps.figure()

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))